In [ ]:
%load_ext sql

In [ ]:
%run create_tables.py

In [ ]:
%run etl.py

In [ ]:
"""
Create a connection to the sparkifydw
"""
import psycopg2
config = configparser.ConfigParser()
config.read('dwh.cfg')

DWH_DB= config.get("CLUSTER","DB_NAME")
DWH_DB_USER= config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD= config.get("CLUSTER","DB_PASSWORD")
DWH_PORT = config.get("CLUSTER","DB_PORT")
DWH_ENDPOINT = config.get("CLUSTER","HOST")

conn="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB);

%sql $conn



In [ ]:
"""Validation Check - Get's count of all of the tables"""

In [ ]:
%%sql

select count(*), 'users' as tablename from sparkifydw.users
union
select count(*), 'artists' from sparkifydw.artists
union
select count(*), 'songs' from sparkifydw.songs
union
select count(*), 'time' from sparkifydw.time
union
select count(*), 'songplays' from sparkifydw.songplays
union
select count(*), 'staging_events' from sparkifydw.staging_events
union
select count(*), 'staging_songs' from sparkifydw.staging_songs;


In [ ]:
"""
#### Sample DashBoards Part 1 ####

Load sample queries needed to create the sample dashboards
The database credentials are load from dwh.cfg congfile
The query are populated in different variables and loaded into different dataframes
"""

import configparser
import psycopg2
import matplotlib.pyplot as plt
import pandas as pd

config = configparser.ConfigParser()
config.read('dwh.cfg')

DWH_DB= config.get("CLUSTER","DB_NAME")
DWH_DB_USER= config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD= config.get("CLUSTER","DB_PASSWORD")
DWH_PORT = config.get("CLUSTER","DB_PORT")
DWH_ENDPOINT = config.get("CLUSTER","HOST")

conn="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
qryTotalSongs="""
select * from (
select count(*) total, title from sparkifydw.songplays sp 
    inner join sparkifydw.songs s
        on sp.song_id = s.song_id
group by  title) x
where total >= 5 
order by total
"""

qryTotalUsersByGender ="""
select count(*) total, gender from sparkifydw.users
group by gender;
"""

qryTotalByLevel ="""
select count(*) total, level from sparkifydw.users
group by level
"""

qryLevelByGender="""
select count(*) total, decode(gender,'M','Male','F','Female') || ' - ' ||  level genderbylevel from sparkifydw.users
group by gender, level
order by 1;
"""

stats_song = pd.read_sql_query(qryTotalSongs,conn)
stats_userbygender =  pd.read_sql_query(qryTotalUsersByGender,conn)
stats_totalbylevel =  pd.read_sql_query(qryTotalByLevel,conn)
stats_levelbygender = pd.read_sql_query(qryLevelByGender,conn)

In [ ]:
"""
#### Sample DashBoards Part 2 ####
Part 2 builds the dashboards based on the queries of Part 1
"""

# Most Song Played

plt.bar(stats_song.total, stats_song.title)
plt.title('Most Song Played')
plt.ylabel('Songs')
plt.xlabel('Number Of Plays')
plt.show()

# Total Users by Gender
plt.barh(stats_userbygender.gender, stats_userbygender.total)
plt.title('Total Users by Gender')
plt.ylabel('Gender')
plt.xlabel('Total')
plt.show()

# Total by levels
fig1, ax1 = plt.subplots()
ax1.pie(stats_totalbylevel.total, labels=stats_totalbylevel.level, explode=None,  autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.show()

# Total level by Gender 
fig1, ax1 = plt.subplots()
ax1.pie(stats_levelbygender.total, labels=stats_levelbygender.genderbylevel, explode=None,  autopct= '%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.show()